In [1]:
from pyspark.sql import SparkSession, functions as f

In [2]:
spark = (
    SparkSession.builder.appName("Preprocessing_Yellow")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '3g')   
    .config('spark.executor.memory', '4g')  
    .config('spark.executor.instances', '2')  
    .config('spark.executor.cores', '2')
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
23/09/08 18:48:11 WARN Utils: Your hostname, DulanComputer resolves to a loopback address: 127.0.1.1; using 172.30.15.25 instead (on interface eth0)
23/09/08 18:48:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/08 18:48:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/08 18:48:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
orders = spark.read.parquet("../../data/insights/pre_insights/orders.parquet/")

In [4]:
orders.dtypes

[('merchant_abn', 'bigint'),
 ('merchant_name', 'string'),
 ('consumer_id', 'bigint'),
 ('order_datetime', 'date'),
 ('dollar_value', 'double')]

In [5]:
orders.show(truncate= False)

+------------+------------------------------+-----------+--------------+------------------+
|merchant_abn|merchant_name                 |consumer_id|order_datetime|dollar_value      |
+------------+------------------------------+-----------+--------------+------------------+
|32361057556 |Orci In Consequat Corporation |1058499    |2021-05-20    |97.59893835228833 |
|32361057556 |Orci In Consequat Corporation |1058499    |2021-05-20    |97.59893835228833 |
|43186523025 |Lorem Ipsum Sodales Industries|1058499    |2021-07-15    |54.12123684966841 |
|43186523025 |Lorem Ipsum Sodales Industries|1058499    |2021-07-15    |54.12123684966841 |
|50315283629 |Iaculis Aliquet Diam LLC      |1058499    |2021-05-06    |209.03399492699737|
|50315283629 |Iaculis Aliquet Diam LLC      |1058499    |2021-05-06    |209.03399492699737|
|87630626808 |Eget Industries               |1058499    |2021-05-06    |38.08559535222015 |
|87630626808 |Eget Industries               |1058499    |2021-05-06    |38.08559

In [6]:
orders_aggregated = orders.groupBy("merchant_abn").agg\
                        (f.count("order_datetime").alias("number_of_orders"),
                         f.avg("dollar_value").alias("average_cost_of_order"))

In [7]:
orders_aggregated.show(truncate=False)

+------------+----------------+---------------------+
|merchant_abn|number_of_orders|average_cost_of_order|
+------------+----------------+---------------------+
|83412691377 |21889           |34.71684053809685    |
|19839532017 |1140            |157.0                |
|38700038932 |11253           |1339.5719373318022   |
|15613631617 |2743            |301.09223404688345   |
|73256306726 |8031            |285.62311107912535   |
|73841664453 |1557            |84.58861376785562    |
|60654402457 |287             |85.95947863687985    |
|96946925998 |173             |1008.0150573862218   |
|35344855546 |2269            |86.80869363397993    |
|34440496342 |346             |90.72410719716241    |
|52763133264 |211             |117.29243205045037   |
|12516851436 |331             |156.69603561548612   |
|78916025936 |86              |327.7552181457143    |
|48214071373 |844             |295.2395055486349    |
|38986645707 |63              |1073.9954774430323   |
|41956465747 |447           

In [8]:
orders_aggregated.write.mode("overwrite").parquet("../../data/insights/agg_insight_data/orders_agg.parquet")

In [9]:
spark.stop()